In [1]:
import time
from binance.client import Client
import mysql.connector
import pandas as pd
import threading

client = Client()

# List of coins to fetch data for
coins = ["sxp", "chess", "blz", "joe", "perl", "ach", "gmt", "xrp", "akro", "zil", "cfx", "adx", "chz", "bel", "alpaca", "elf", "epx", "pros", "t", "dar", "agix", "mob", "id", "trx", "key", "tru", "amb", "magic", "lina", "lever"]

def get_db_connection():
    connection = mysql.connector.connect(
        host='68.178.153.6',
        user='ran_db',
        password='Ahsan@123',
        database='ran_dbb'
    )
    return connection

# Create tables for each coin
def create_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    for coin in coins:
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {coin}usdt (
                id INT AUTO_INCREMENT PRIMARY KEY,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                price DECIMAL(18, 8),
                volume DECIMAL(18, 8)
            )
        """)
    connection.commit()
    connection.close()

def fetch_and_store_recent_trades(coin):
    last_trade_timestamp = None
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            trades = client.get_recent_trades(symbol=f'{coin.upper()}USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['time'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(f"INSERT INTO {coin}usdt (timestamp, price, volume) VALUES (%s, %s, %s)",
                                   (trade_timestamp, trade['price'], trade['qty']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin}: {e}')
            connection.rollback()
            # Wait for 30 seconds before retrying
            time.sleep(30)
            continue
        finally:
            connection.close()
        time.sleep(5)

# Function to delete data older than 7 days
def delete_old_data(coin):
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()

            # Deleting in batches of 1000 rows to avoid large transactions
            rows_deleted = 1  # Initialize with a dummy value to enter the loop
            while rows_deleted > 0:
                cursor.execute(f"""
                    DELETE FROM {coin}usdt
                    WHERE timestamp < DATE_SUB(NOW(), INTERVAL 7 DAY)
                    LIMIT 1000
                """)
                rows_deleted = cursor.rowcount
                connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            if connection:
                connection.rollback()
        finally:
            if connection:
                connection.close()

        # Sleep for a certain period before running the deletion again. This example uses 6 hours.
        time.sleep(21600)  # Sleep for 6 hours

# Start the threads
def start_threads():
    create_tables()
    for coin in coins:
        threading.Thread(target=fetch_and_store_recent_trades, args=(coin,)).start()
        threading.Thread(target=delete_old_data, args=(coin,)).start()

start_threads()

while True:
    time.sleep(30)


Exception in thread Thread-49:
Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\mysql\connector\connection_cext.py", line 291, in _open_connection
    self._cmysql.connect(**cnx_kwargs)
_mysql_connector.MySQLInterfaceError: User ran_db already has more than 'max_user_connections' active connections

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_18792\2732027514.py", line 41, in fetch_and_store_recent_trades
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_18792\2732027514.py", line 13, in get_db_connection
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\mysql\connector\pooling.py", line 293, in connect
    return CMySQLConnection(*args, **kwargs)
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\mysql\connector\connection_cext.py", line 120, in __init__
    self.connect(**kwargs)
  File "c:\Users\Ask78\a

Error fetching and storing trades for id: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for magic: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for pros: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error fetching and storing trades for agix: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for chz: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error fetching and storing trades for trx: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for trx: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for t: 1203 (42000): User ran_db already has more than 'max_user_connections' acti

    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_18792\2732027514.py", line 58, in fetch_and_store_recent_trades
UnboundLocalError: local variable 'connection' referenced before assignment
Exception in thread Thread-61:
Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\mysql\connector\connection_cext.py", line 291, in _open_connection
Exception in thread Thread-58:
Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\mysql\connector\connection_cext.py", line 291, in _open_connection
    self._cmysql.connect(**cnx_kwargs)
_mysql_connector.MySQLInterfaceError: User ran_db already has more than 'max_user_connections' active connections

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_18792\2732027514.py", line 41, in fetch_and_store_recent_trades

Error deleting old data for amb: 1203 (42000): User ran_db already has more than 'max_user_connections' active connectionsError fetching and storing trades for lina: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections

Error deleting old data for elf: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for epx: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error fetching and storing trades for pros: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error fetching and storing trades for dar: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error deleting old data for id: 1203 (42000): User ran_db already has more than 'max_user_connections' active connections
Error fetching and storing trades for ach: 1203 (42000): User ran_db already has more than 'max_user_connect